In [1]:
from fastai2.vision.all import *

# Checar el VerboseCallback

In [2]:
from fastai2.test_utils import VerboseCallback

In [3]:
class VerboseCallback(Callback):
    "Callback that prints the name of each event called"
    def __call__(self, event_name):
        print(event_name)
        super().__call__(event_name)

# Crear callback que anote los tiempos del learner

In [7]:
import threading
import time
from torch.autograd.profiler import FunctionEvent, EventList

class ChromeStatics(Callback):
    def __init__(self, ll):
        super().__init__()
        self.l = ll
        self.d = {}
    def __call__(self, event_name):
        tid = threading.get_ident()
        tname = threading.current_thread().name
        if event_name.startswith('begin'):
            evt = event_name.replace('begin_', '')
            self.d[evt] = time.clock_gettime_ns(time.CLOCK_PROCESS_CPUTIME_ID)/1000
        elif event_name.startswith('after'):
            evt = event_name.replace('after_', '')
            end = time.clock_gettime_ns(time.CLOCK_PROCESS_CPUTIME_ID)/1000
            if evt in self.d:
                start = self.d[evt]
                fe = FunctionEvent(tname, tid, evt, tid, start, end)
            else:
                print(f"ENDDDD before 500 {evt}")
                start = end-500 # 1-time after_xxx
                fe = FunctionEvent(tname, tid, evt, tid, start, end)
            self.l.append(fe)
            self.d.pop(evt, None)
    def export_chrome_trace(self, fname="export_chrome_trace.json", dumps=False):
        with open(fname, mode="w+") as f:
            prof = EventList(self.l)
            prof.export_chrome_trace(f.name)
            # Now validate the json
            if dumps:
                parsed = json.load(f)
                print(json.dumps(parsed, indent=4, sort_keys=True))

# Usarlo en ejemplo sencillo

In [8]:
%%time
from fastai2.vision.all import *
path = untar_data(URLs.PETS)
files = get_image_files(path/"images")
def label_func(f): return f[0].isupper()
dls = ImageDataLoaders.from_name_func(path, files, label_func, item_tfms=Resize(224))
learn = cnn_learner(dls, resnet34, metrics=error_rate)
my_list = []
cb_stats = ChromeStatics(my_list)
learn.fine_tune(1, cbs=cb_stats)
cb_stats.export_chrome_trace()

epoch,train_loss,valid_loss,error_rate,time
0,0.160505,0.030661,0.006766,00:19


epoch,train_loss,valid_loss,error_rate,time
0,0.058074,0.030120,0.007442,00:20


CPU times: user 16.9 s, sys: 2.58 s, total: 19.5 s
Wall time: 41.6 s


# profile autograd de pytorch

In [ ]:
%%time
from fastai2.vision.all import *
path = untar_data(URLs.PETS)
files = get_image_files(path/"images")
def label_func(f): return f[0].isupper()
dls = ImageDataLoaders.from_name_func(path, files, label_func, item_tfms=Resize(224))
learn = cnn_learner(dls, resnet34, metrics=error_rate)
my_list = []
cb_stats = ChromeStatics(my_list)
with torch.autograd.profiler.profile() as prof:
    learn.fine_tune(1, cbs=cb_stats)
    cb_stats.export_chrome_trace()
prof.export_chrome_trace("prof.json")

In [ ]:
type(prof), type(cb_stats)

In [ ]:
with open('prof.json', 'r') as p:
    with open('export_chrome_trace.json', 'r') as e:
        jsonp = json.load(p)
        jsone = json.load(e)
        r = jsonp+jsone
        with open('mezcla.json', mode="w+") as fout:
            fout.write(json.dumps(r))
print('done')

# change learner loop

In [6]:
%%time
from fastai2.vision.all import *

@patch_to(Learner)
def begin_pred(self:Learner): return
@patch_to(Learner)
def one_batch(self, i, b):
        self.iter = i
        try:
            print("----------- 1")
            self._split(b);                                  self('begin_batch')
            print("----------- (((((((((((((((2)))))))))))))))")
            self('begin_pred')
            print("----------- (((3)))")
            self.pred = self.model(*self.xb);                self('after_pred')
            print("----------- 4")
            if len(self.yb) == 0: return
            print("----------- 5")
            #self('begin_loss')
            print("----------- 6")
            self.loss = self.loss_func(self.pred, *self.yb); self('after_loss')
            print("----------- 7")
            if not self.training: return
            print("----------- 8")
            #self('begin_backward')
            print("----------- 9")
            self.loss.backward();                            self('after_backward')
            print("----------- 10")
            self.opt.step();                                 self('after_step')
            print("----------- 11")
            self.opt.zero_grad()
        except CancelBatchException:                         self('after_cancel_batch')
        finally:                                             self('after_batch')

            
path = untar_data(URLs.PETS)
files = get_image_files(path/"images")
def label_func(f): return f[0].isupper()
dls = ImageDataLoaders.from_name_func(path, files, label_func, item_tfms=Resize(224))
learn = cnn_learner(dls, resnet34, metrics=error_rate)
my_list = []
cb_stats = ChromeStatics(my_list)
learn.fine_tune(1, cbs=cb_stats)
cb_stats.export_chrome_trace()

              begin_fit


              begin_epoch
              begin_train
----------- 1
              begin_batch
----------- (((((((((((((((2)))))))))))))))
              after_batch
              after_train
              after_epoch
              after_fit


AssertionError: 